# Convertible Bond Valuation - QL Example

Value a Convertible Bond and compare to QL example at http://gouthamanbalaraman.com/blog/value-convertible-bond-quantlib-python.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.bonds import *
from financepy.market.curves import *

####################################################################
# FINANCEPY BETA Version 0.180 - This build:  22 Sep 2020 at 22:58 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#    For bug reports and comments - allmypythonprojects@gmail.com  #
####################################################################



## Creating the Convertible Bond

In [3]:
valuationDate = FinDate(9, 1, 2004)
settlementDate = valuationDate.addDays(2)

We set the maturity date, annualised coupon, coupon frequency and convention for calculating accrued interest

In [4]:
maturityDate = FinDate(15, 3, 2022)
coupon = 0.0575
frequencyType = FinFrequencyTypes.SEMI_ANNUAL
accrualBasis = FinDayCountTypes.THIRTY_360_BOND
face = 100.0 
conversionRatio = 3.84615  

We need to set the face amount and the conversion ratio. This is the number of shares you can convert to for that face amount.

We can also delay the start of the conversion feature using the start convert date. We allow conversion to occur from settlement.

In [5]:
startConvertDate = settlementDate

We can also set the call schedule - the price and date on which the issuer can call back the bond. This is given with reference to a price of 100. Making the call price very high means it will never be called. So the call has no impact.

In [6]:
callPrice = 100.0
callDates = [FinDate(20, 3, 2007), FinDate(15, 3, 2012), FinDate(15, 3, 2017)]
callPrices = [callPrice, callPrice, callPrice]

We can also set the put schedule - the price and date on which the owner can put back the bond to the issuer. This is given with reference to a price of 100. Making the put price very low means it will never be put.

In [7]:
putPrice = 100.0
putDates = [FinDate(20, 3, 2007), FinDate(15, 3, 2012), FinDate(15, 3, 2017)]
putPrices = [putPrice, putPrice, putPrice]

At this point we can create our bond.

In [8]:
bond = FinBondConvertible(maturityDate, coupon, frequencyType, startConvertDate, conversionRatio,
                          callDates, callPrices, putDates, putPrices, accrualBasis, face)

In [9]:
print(bond)

OBJECT TYPE: FinBondConvertible
MATURITY DATE: TUE 15 MAR 2022
COUPON: 0.0575
FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: FinDayCountTypes.THIRTY_360_BOND
FACE AMOUNT: 100.0
CONVERSION RATIO: 3.84615
START CONVERT DATE: SUN 11 JAN 2004
CALL: DATES
TUE 20 MAR 2007: 100.0
THU 15 MAR 2012: 100.0
WED 15 MAR 2017: 100.0
PUT: DATES
TUE 20 MAR 2007: 100.0
THU 15 MAR 2012: 100.0
WED 15 MAR 2017: 100.0



## Valuation Inputs

The model allows a discrete dividend schedule. We can use the FinSchedule class to do this.

In [10]:
dividendSchedule = FinSchedule(settlementDate, maturityDate).scheduleDates()[1:]
dividendYields = [0.02] * len(dividendSchedule)

In [13]:
rate = 0.04
discountCurve = FinDiscountCurveFlat(settlementDate, rate, FinFrequencyTypes.CONTINUOUS)

In [18]:
stockPrice = 29.04
stockVolatility = 0.40
creditSpread = 0.03
recoveryRate = 0.40 # 40 percent
numStepsPerYear = 1000

In [19]:
bond.value(valuationDate, stockPrice, stockVolatility, dividendSchedule, dividendYields,
           discountCurve, creditSpread, recoveryRate, numStepsPerYear)

{'cbprice': 135.35429314756908,
 'bond': 89.47598110542303,
 'delta': 2.8961224798738208,
 'gamma': 0.5951936335206977,
 'theta': 221.66604283982775}

Copyright (c) 2020 Dominic O'Kane

This compares to 132.31 found by QL. The difference could be due to the different treatment of dividends or the credit spread.